# Tutorial: Accelerating pygpc by using differen computing backends

This tutorial shows how to accelerate pygpc by choosing different computing backends. At the moment the following backends are available:
1. Implementation in Python, name in pygpc: **python**
2. Implementation in C++, name in pygpc: **cpu**
3. Implementation in C++ and OpenMP, name in pygpc: **omp**
4. Implementation in CUDA-C, name in pygpc: **cuda**, requires an Nvidia GPU

# Let's get started

## First, we set up our gpc

As a model we are going to use our good old three-dimensional test function of Ishigami.
$$
y = \sin(x_1) + a \sin(x_2)^2 + b x_3^4 \sin(x_1)
$$

In [10]:
import pygpc
import numpy as np
from collections import OrderedDict

fn_results = "tmp/gpc"

# define problem (the parameter names have to be the same as in the model)
parameters = OrderedDict()
parameters["x1"] = pygpc.Beta(pdf_shape=[1, 1], pdf_limits=[-np.pi, np.pi])
parameters["x2"] = pygpc.Beta(pdf_shape=[1, 1], pdf_limits=[-np.pi, np.pi])
parameters["x3"] = 0.
parameters["a"] = 7.
parameters["b"] = 0.1

# define model
model = pygpc.testfunctions.Ishigami().set_parameters(parameters)

# define problem
problem = pygpc.Problem(model, parameters)

# gPC options
options = dict()
options["order_start"] = 5
options["order_end"] = 20
options["solver"] = "LarsLasso"
options["interaction_order"] = 2
options["order_max_norm"] = 0.7
options["n_cpu"] = 0
options["adaptive_sampling"] = True
options["gradient_enhanced"] = True
options["fn_results"] = fn_results
options["eps"] = 0.0075

# define algorithm
algorithm = pygpc.RegAdaptive(problem=problem, options=options)

# Initialize gPC Session
session = pygpc.Session(algorithm=algorithm)

# run gPC session
session, coeffs, results = session.run()

Initializing gPC object...
Initializing gPC matrix...
Order/Interaction order: 5/2
Starting adaptive sampling:
Extending grid from 14 to 14 by 0 sampling points
Performing simulations 1 to 14
It/Sub-it: 5/2 Performing simulation 01 from 14 [==                                      ] 7.1%
Total parallel function evaluation: 0.006322383880615234 sec
It/Sub-it: 5/2 Performing simulation 01 from 28 [=                                       ] 3.6%
Gradient evaluation: 0.0017218589782714844 sec
Determine gPC coefficients using 'LarsLasso' solver (gradient enhanced)...
LOOCV 01 from 14 [==                                      ] 7.1%
LOOCV 02 from 14 [=====                                   ] 14.3%
LOOCV 03 from 14 [========                                ] 21.4%
LOOCV 04 from 14 [===========                             ] 28.6%
LOOCV 05 from 14 [==============                          ] 35.7%
LOOCV 06 from 14 [=================                       ] 42.9%
LOOCV 07 from 14 [====================

LOOCV 01 from 18 [==                                      ] 5.6%
LOOCV 02 from 18 [====                                    ] 11.1%
LOOCV 03 from 18 [======                                  ] 16.7%
LOOCV 04 from 18 [========                                ] 22.2%
LOOCV 05 from 18 [===========                             ] 27.8%
LOOCV 06 from 18 [=============                           ] 33.3%
LOOCV 07 from 18 [===============                         ] 38.9%
LOOCV 08 from 18 [=================                       ] 44.4%
LOOCV 09 from 18 [====================                    ] 50.0%
LOOCV 10 from 18 [======================                  ] 55.6%
LOOCV 11 from 18 [========================                ] 61.1%
LOOCV 12 from 18 [==========================              ] 66.7%
LOOCV 13 from 18 [============================            ] 72.2%
LOOCV 14 from 18 [===============================         ] 77.8%
LOOCV 15 from 18 [=================================       ] 83.3%
LOOCV 16 fr

LOOCV 13 from 22 [=======================                 ] 59.1%
LOOCV 14 from 22 [=========================               ] 63.6%
LOOCV 15 from 22 [===========================             ] 68.2%
LOOCV 16 from 22 [=============================           ] 72.7%
LOOCV 17 from 22 [==============================          ] 77.3%
LOOCV 18 from 22 [================================        ] 81.8%
LOOCV 19 from 22 [==================================      ] 86.4%
LOOCV 20 from 22 [====================================    ] 90.9%
LOOCV 21 from 22 [======================================  ] 95.5%
LOOCV 22 from 22 [========================================] 100.0%
LOOCV computation time: 0.2890756130218506 sec
-> relative loocv error = 0.5257820123178282
Extending grid from 22 to 23 by 1 sampling points
Performing simulations 23 to 23
It/Sub-it: 6/1 Performing simulation 1 from 1 [========================================] 100.0%
Total parallel function evaluation: 0.009192705154418945 sec
It/Sub-i

LOOCV 13 from 25 [====================                    ] 52.0%
LOOCV 14 from 25 [======================                  ] 56.0%
LOOCV 15 from 25 [========================                ] 60.0%
LOOCV 16 from 25 [=========================               ] 64.0%
LOOCV 17 from 25 [===========================             ] 68.0%
LOOCV 18 from 25 [============================            ] 72.0%
LOOCV 19 from 25 [==============================          ] 76.0%
LOOCV 20 from 25 [================================        ] 80.0%
LOOCV 21 from 25 [=================================       ] 84.0%
LOOCV 22 from 25 [===================================     ] 88.0%
LOOCV 23 from 25 [====================================    ] 92.0%
LOOCV 24 from 25 [======================================  ] 96.0%
LOOCV 25 from 25 [========================================] 100.0%
LOOCV computation time: 0.5119976997375488 sec
-> relative loocv error = 0.721638451577832
Extending grid from 25 to 27 by 2 sampling points


LOOCV 04 from 25 [======                                  ] 16.0%
LOOCV 05 from 25 [========                                ] 20.0%
LOOCV 06 from 25 [=========                               ] 24.0%
LOOCV 07 from 25 [===========                             ] 28.0%
LOOCV 08 from 25 [============                            ] 32.0%
LOOCV 09 from 25 [==============                          ] 36.0%
LOOCV 10 from 25 [================                        ] 40.0%
LOOCV 11 from 25 [=================                       ] 44.0%
LOOCV 12 from 25 [===================                     ] 48.0%
LOOCV 13 from 25 [====================                    ] 52.0%
LOOCV 14 from 25 [======================                  ] 56.0%
LOOCV 15 from 25 [========================                ] 60.0%
LOOCV 16 from 25 [=========================               ] 64.0%
LOOCV 17 from 25 [===========================             ] 68.0%
LOOCV 18 from 25 [============================            ] 72.0%
LOOCV 19 f

It/Sub-it: 7/2 Performing simulation 1 from 2 [====================                    ] 50.0%
Total parallel function evaluation: 0.007985591888427734 sec
It/Sub-it: 7/2 Performing simulation 1 from 4 [==========                              ] 25.0%
Gradient evaluation: 0.0019526481628417969 sec
Determine gPC coefficients using 'LarsLasso' solver (gradient enhanced)...
LOOCV 01 from 25 [=                                       ] 4.0%
LOOCV 02 from 25 [===                                     ] 8.0%
LOOCV 03 from 25 [====                                    ] 12.0%
LOOCV 04 from 25 [======                                  ] 16.0%
LOOCV 05 from 25 [========                                ] 20.0%
LOOCV 06 from 25 [=========                               ] 24.0%
LOOCV 07 from 25 [===========                             ] 28.0%
LOOCV 08 from 25 [============                            ] 32.0%
LOOCV 09 from 25 [==============                          ] 36.0%
LOOCV 10 from 25 [================ 

Determine gPC coefficients using 'LarsLasso' solver (gradient enhanced)...
LOOCV 01 from 25 [=                                       ] 4.0%
LOOCV 02 from 25 [===                                     ] 8.0%
LOOCV 03 from 25 [====                                    ] 12.0%
LOOCV 04 from 25 [======                                  ] 16.0%
LOOCV 05 from 25 [========                                ] 20.0%
LOOCV 06 from 25 [=========                               ] 24.0%
LOOCV 07 from 25 [===========                             ] 28.0%
LOOCV 08 from 25 [============                            ] 32.0%
LOOCV 09 from 25 [==============                          ] 36.0%
LOOCV 10 from 25 [================                        ] 40.0%
LOOCV 11 from 25 [=================                       ] 44.0%
LOOCV 12 from 25 [===================                     ] 48.0%
LOOCV 13 from 25 [====================                    ] 52.0%
LOOCV 14 from 25 [======================                  ] 56.0%
LOO

LOOCV 22 from 25 [===================================     ] 88.0%
LOOCV 23 from 25 [====================================    ] 92.0%
LOOCV 24 from 25 [======================================  ] 96.0%
LOOCV 25 from 25 [========================================] 100.0%
LOOCV computation time: 0.46521520614624023 sec
-> relative loocv error = 0.05873950652966923
Extending grid from 49 to 51 by 2 sampling points
Performing simulations 50 to 51
It/Sub-it: 9/2 Performing simulation 1 from 2 [====================                    ] 50.0%
Total parallel function evaluation: 0.008755207061767578 sec
It/Sub-it: 9/2 Performing simulation 1 from 4 [==========                              ] 25.0%
Gradient evaluation: 0.0018346309661865234 sec
Determine gPC coefficients using 'LarsLasso' solver (gradient enhanced)...
LOOCV 01 from 25 [=                                       ] 4.0%
LOOCV 02 from 25 [===                                     ] 8.0%
LOOCV 03 from 25 [====                                  

LOOCV 21 from 25 [=================================       ] 84.0%
LOOCV 22 from 25 [===================================     ] 88.0%
LOOCV 23 from 25 [====================================    ] 92.0%
LOOCV 24 from 25 [======================================  ] 96.0%
LOOCV 25 from 25 [========================================] 100.0%
LOOCV computation time: 0.4348132610321045 sec
-> relative loocv error = 0.01035686936252513
Extending grid from 53 to 56 by 3 sampling points
Performing simulations 54 to 56
It/Sub-it: 10/2 Performing simulation 1 from 3 [=============                           ] 33.3%
Total parallel function evaluation: 0.010661125183105469 sec
It/Sub-it: 10/2 Performing simulation 1 from 6 [======                                  ] 16.7%
Gradient evaluation: 0.0025281906127929688 sec
Determine gPC coefficients using 'LarsLasso' solver (gradient enhanced)...
LOOCV 01 from 25 [=                                       ] 4.0%
LOOCV 02 from 25 [===                                 

The default backend is the **omp**-backend using OpenMP